<img src="images/steno3d.png" width="200"></img>
<br>
<h1 style="color:#000080;"> Line Example: Tsyganenko Magnetic Field </h1> 

<p> This example shows the <b>Earth's Magnetosphere</b> based on <a href=http://ccmc.gsfc.nasa.gov/models/modelinfo.php?model=Tsyganenko%20Magnetic%20Field> Tsyganenko Magnetic Field Model</a>. The simulated geomagnetic field is shown here as streamlines which are generated from selected points in <a href=http://www.mssl.ucl.ac.uk/grid/iau/extra/local_copy/SP_coords/geo_sys.htm>Geocentric Solar Ecliptic (GSE)</a> coordinate system. Because the magnetic field consists of 3D vectors over continous spatial domain we can use lines to show the general trend of the data.

<p> This document provides the detailed procedure to uploaded data in <a href="https://steno3d.com">Steno3D</a> and visualize magnetic field lines.</p>

<div>
    <figure>
    <div style="height: 300px;">
    <img src="images/mag1.png" style="float: left; width: 48%; margin-top: 5%">
    <img src="images/mag2.png" style="float: right; width: 48%; margin-top: 5%">
    </div>
    <figcaption align="middle"><div style="width: 55%; margin-left: auto; margin-right: auto; font-weight: bold; font-style: italic; font-size: 15px; margin-top: 20%; margin-bottom: 3%"> Figure 1: Visualization of Tsyganenko Magnetic Field model in Steno3D </div>
    </figcaption>
    </figure>
</div>

<h3 style="color:#000080;"> Import Dependencies </h3>

In [ ]:
import numpy as np
import steno3d

<h3 style="color:#000080;"> Login to Steno3D </h3>

Before using Steno3D, you need to <a href="https://steno3d.com/signup">sign up for an account</a> to get your own developer API key if you do not have one already. Running the cell below will provide you with instructions for how to obtain and enter your key. 

The option to `skip_credentials` prevents saving your developer key to the mybinder.org server. When working on your local computer, remove this argument and your key will be saved.

In [ ]:
steno3d.login(skip_credentials=True)

<h3 style="color:#000080;"> Steno3D Lines </h3>

Before moving to the actual example, this section briefly describes the process of drawing lines in Steno3D. The way that Steno3D works is to first create a mesh and then bind the data to the mesh. The mesh defines the geometry of the line. So the first step to create lines is to make a 1D mesh. 1D meshes are formed by vertices and segments which define how the vertices are connected. A simple example of a line mesh is depicted in the Figure 2.

<figure>
    <img src="images/mesh1d.png" width ="950px" height="500px" align= middle>
    <figcaption align="middle"><div style="width: 30%; margin: auto; font-weight: bold; font-style: italic; font-size: 15px; margin-bottom: 5%; margin-top: 3%"> Figure 2: Line Mesh (Mesh1D)</div> </figcaption>
</figure>

As shown in this figure, vertices are 3D (x, y, z) points and segments consists of two integers defining which vertices make up that segment. Note that there is no segment [2,3] which shows the end of line one and the start of line two are not connected. Vertices must always be a n x 3 array of spatial coordinates while segments must be an array of vertex indices with a dimension of m x 2. Additional documentation for Mesh1D can be found <a href="https://python.steno3d.com/en/latest/content/api/resources/line.html#meshes">online</a>.

The next step after defining a mesh is to bind the data. We can bind the data either to the nodes 'N' (vertices) or cell centers 'CC' (segments). API documentation is available <a href="https://python.steno3d.com/en/latest/content/api/resources/data.html#resources-data">here</a>, but we explain these steps in detail while drawing the magnetic field lines in the following sections.

<h3 style="color:#000080;"> Magnetic Field Data </h3>

For this notebook, we will be using example data from the `steno3d.examples.Tsyganenko` module. This module downloads raw data, saves it to a folder in the home directory (or another location you may specify), and allows easy access to all the different pieces of data. The data contains the following information:
<ul>
 <li><b>X, Y, Z</b>: GSE coordinates, Re</li>
 <li><b>B</b>: Magnitude of magnetic field at selected point in 3D space, nT</li>
</ul>

Where <b>Re</b> is an Earth-radius unit (approximately equal to 6371 km) and <b>nT</b> is nanotesla.

In [ ]:
from steno3d.examples import Tsyganenko
Tsyganenko.fetch_data(verbose=False)  # To specify a different data directory than the
                                      # default '$HOME/.steno3d_python_assets/' use
                                      # keyword argument `directory='/path/to/your/directory/'`

In this section we are parsing the data file to create the vertices, segments, and magnetic field (B) values at the vertices in 3D space. 

In [ ]:
verts = []
segs = []
data_b = []

with open(Tsyganenko.datafile, 'r') as f:
    for line in f:
        items = line.split()
        verts.append([float(i) for i in items[1:4]])
        data_b.append(float(items[4]))
        if items[0] != '1':
            segs.append([len(verts)-2, len(verts)-1])

<h3 style="color:#000080;"> Create a Project </h3>

Before making any resources (points, lines, surfaces or volumes) in Steno3D, we need to create a `Project` to assign the resource to that project. Projects organize all your resources together and can contain a combination of resources; they can also be viewed and shared in Steno3D. 

Following are the properties of projects that you can modify if desired:

<ul>
    <li><b>title</b>: Title of the project</li>
    <li><b>description</b>: Description of the project</li>
    <li><b>public</b>: If True, the project is viewable by anyone online. If False, the project is private and only viewable by people you choose to share it with. Free and Academic accounts have limits on the number of private projects allowed; if you would like more, upgrade your account at steno3d.com.</li>
    <li><b>resources</b>: List of resources the project contains.</li>
</ul>

You can <a href="https://steno3d.com/explore">explore public projects on steno3d.com.</a> Additional <a href="https://python.steno3d.com/en/latest/content/api/projects.html">API documentation</a> is available online.

In [ ]:
magnetic_field_proj = steno3d.Project(
    title='Tsyganenko Magnetic Field',
    description="a simple model of the Earth's magnetic field",
    public=False
)

<h3 style="color:#000080;"> Create Lines </h3>

Now that we have the segs, vertices and `Project` we are ready to make the magnetic field lines. 
The properties of line are:

<ul>
    <li><b>project</b> (required)<sup><b>*</b></sup>: The project(s) that contain the resource</li>
    <li><b>mesh</b>: Lines use `Mesh1D`, a mesh that contains verices and segments </li>
    <li><b>data</b>: A list of `DataArrays` bound to the mesh </li>
    <li><b>opts</b>: A dictionary of options (available options can be found in <a href="https://python.steno3d.com/en/latest/content/api/resources/line.html#options">API documentation</a>)</li>
    <li><b>title</b></li>
    <li><b>description</b></li>
</ul>

(*): For the Volume resource to pass validation prior to upload, required properties must be set.

Here, we initialize a line with the containing `project` and a `title`. Geometry and options will be added subsequently. 

Additional <a href="https://python.steno3d.com/en/latest/content/api/resources/line.html">API documentation</a> is available online.

In [ ]:
magnetic_line = steno3d.Line(
    project=magnetic_field_proj,
    title="Earth's magnetic field lines"
)

<h3 style="color:#000080;"> Line Mesh </h3>

It is time to make a mesh for the line. The mesh used with lines is a Mesh1D. These meshes have the following properties:

<ul>
    <li><b>vertices</b>(required): n x 3 array of spatial coordinates</li>
    <li><b>segments</b>(required): m x 2 array of vertex indices connecting each segment</li>
    <li><b>opts</b>: The only available option is `view_type`, set to `line` or `tube` with a dictionary</li>
    <li><b>title</b></li>
    <li><b>description</b></li>
</ul>

Mesh1D API documentation is available within the <a href="https://python.steno3d.com/en/latest/content/api/resources/line.html#meshes">line documentation</a>.

In [ ]:
magnetic_line_mesh = steno3d.Mesh1D(
    vertices=np.array(verts),
    segments=np.array(segs),
    opts=dict(view_type='line')
)

Finally, this mesh should be added to the line. 

In [ ]:
magnetic_line.mesh = magnetic_line_mesh

<h3 style="color:#000080;"> Binding Line Data </h3>

To bind data to the line, we need 1D array of data. The data are bound to the line with a dictionary containing:
<ul>
    <li><b>location</b> (required): This is 'N' (nodes or vertices) or 'CC' (cell centers or segments)</li>
    <li><b>data</b> (required): the DataArray</li>
    <li><b>description</b></li>
</ul>

There is a one to one ratio between data in the DataArray and vertices for node data or, for cell-center data, between DataArray and segments. API documentation is available for <a href="https://python.steno3d.com/en/latest/content/api/resources/data.html#resources-data">DataArray</a>.

In this example we will bind the data to 'N' location:

In [ ]:
mag_data = steno3d.DataArray(
    array=data_b,
    title='Magnitude of magnetic field (nT)'
)

magnetic_line.data = [
    dict(
        location='N',
        data=mag_data
    )
]

In [ ]:
mag_log_data = steno3d.DataArray(
    array=np.log(data_b),
    title='Natural log of magnitude of magnetic field (log(nT))'
)

magnetic_line.data += [
    dict(
        location='N',
        data=mag_log_data
    )
]

<h3 style="color:#000080;"> Line Options </h3>

We can set the <a href="https://python.steno3d.com/en/latest/content/api/resources/line.html">options</a> for the line. These include color and opacity.

In [ ]:
magnetic_line.opts = dict(
    color=[34, 139, 34],
    opacity=0.5
)

<h3 style="color:#000080;"> Upload and Plot </h3>

Calling `upload` will check that all aspects of line and its containing project are built correctly, then upload the project to <a href="https://steno3d.com">steno3d.com</a>.

In [ ]:
magnetic_field_proj.upload()

Once the project is uploaded, you can view it at the link provided or you can plot the individual line inline here with `plot`.

In [ ]:
magnetic_line.plot()